In [381]:
import psycopg2
import json

In [382]:
request = {
    "entities": {
        "capabilities": [
            "Apache Hadoop",
            "Spark",
            "AWS",
            "arquitecturas de datos escalables",
            "análisis de datos en tiempo real",
            "automatización de procesos",
            "Docker",
            "Kubernetes",
            "Java"
        ],
        "role": "Ingeniero de datos"
    },
    "input_token": 90,
    "output_token": 42,
    "tokens": 132
}

In [383]:
import os
import psycopg2
from psycopg2 import DatabaseError
from dotenv import load_dotenv

#cargar las variables de entorno
load_dotenv()

#Crear conexion con PostgreSQL
def get_db_connection():
    try:
        connection = psycopg2.connect(
            host=os.getenv('PGSQL_HOST'),
            user=os.getenv('PGSQL_USER'),
            password=os.getenv('PGSQL_PASSWORD'),
            dbname=os.getenv('PGSQL_DATABASE'),
            port=os.getenv('PGSQL_PORT')
        )
        return connection
    except DatabaseError as ex:
        raise ex

In [384]:
from psycopg2.extras import RealDictCursor

def buscar_candidatos_postgre(rol, capabilities):
    # Formatear la búsqueda para usar en to_tsquery
    
    busqueda_capabilities = ' | '.join(
            [f"'{capability.strip()}'" if ' ' in capability else capability.strip() for capability in capabilities]
        )

    if busqueda_capabilities.startswith(" | "):
        busqueda_capabilities = busqueda_capabilities[3:]
    if busqueda_capabilities.endswith(" | "):
        busqueda_capabilities = busqueda_capabilities[:-3]
        

    if isinstance(rol, str):
        busqueda_rol = rol.strip()  # Eliminar espacios innecesarios
        busqueda_rol = ' & '.join(busqueda_rol.split())  # Reemplazar espacios por " & " para to_tsquery
    else:
        raise ValueError("El parámetro 'rol' debe ser una cadena.")

    print(busqueda_capabilities, busqueda_rol)
    
    consulta = """
    SET enable_nestloop = off; 
    
    EXPLAIN ANALYZE
    WITH ranked_profiles AS (
    SELECT 
        app_prof.id AS applicant_profile_id,
        u.name AS name,
        skills.description AS skill_description,
        job.name AS job_name,
        level_of_exp.name AS level_of_experience,
        prof_type.description AS profiletype,
        workexp.description AS aditional_info
    FROM 
        recruitment.applicantprofile_skill AS app_skill
    INNER JOIN recruitment.applicant_profile AS app_prof ON app_skill.applicant_profile_id = app_prof.id
    INNER JOIN recruitment.skills AS skills ON skills.id = app_skill.skill_id
    INNER JOIN recruitment.user AS u ON u.user_id = app_prof.user_id
    INNER JOIN recruitment.application AS application ON application.user_id = u.user_id
    INNER JOIN recruitment.job AS job ON job.id = application.job_id
    INNER JOIN recruitment.levelofexp AS level_of_exp ON level_of_exp.id = job.levelofexperience_id
    INNER JOIN recruitment.profiletype AS prof_type ON prof_type.id = job.profile_type_id
    INNER JOIN recruitment.workexperience as workexp ON workexp.applicantprofile_id = app_prof.id
    WHERE 
        -- Usar tsquery para el filtrado en lugar de recalcular el tsvector cada vez
        to_tsvector('spanish', job.name || ' ' || skills.description || ' ' || workexp.description || ' ' || level_of_exp.name)
        @@ to_tsquery('spanish', %s)
        AND
        to_tsvector('spanish', prof_type.description || ' ' || level_of_exp.name) 
        @@ to_tsquery('spanish', %s)
),
ranked_with_row_number AS (
    SELECT 
    applicant_profile_id,
    MIN(name) AS name,
    MIN(skill_description) AS skill_description,
    MIN(job_name) AS job_name,
    MIN(level_of_experience) AS level_of_experience,
    MIN(profiletype) AS profiletype,
    MIN(aditional_info) AS aditional_info
    FROM ranked_profiles
    GROUP BY applicant_profile_id
)
SELECT 
    applicant_profile_id,
    name,
    skill_description,
    job_name,
    level_of_experience,
    profiletype,
    aditional_info
FROM ranked_with_row_number
    """
    resultados = []
    try:
        conn = get_db_connection()  # Obtén la conexión a la base de datos
        with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as cur:
            # Ejecutar la consulta con el término de búsqueda seguro
            cur.execute(consulta, (busqueda_capabilities,busqueda_rol))
            resultados = cur.fetchall()

        conn.close()
    except Exception as e:
        print(f"Error al consultar PostgreSQL: {e}")
    
    # Convertir los resultados en una lista de diccionarios
    # return json.dumps([dict(row) for row in resultados], default=str)
    return [dict(row) for row in resultados]

In [385]:
data = request
role = data["entities"]["role"]
capabilities = data["entities"]["capabilities"]

# Llamar a la función de búsqueda con el término combinado
aplicantes_filtrados = buscar_candidatos_postgre(role, capabilities)


'Apache Hadoop' | Spark | AWS | 'arquitecturas de datos escalables' | 'análisis de datos en tiempo real' | 'automatización de procesos' | Docker | Kubernetes | Java Ingeniero & de & datos


In [386]:
aplicantes_filtrados

[{'QUERY PLAN': 'Finalize GroupAggregate  (cost=31623.73..31636.21 rows=94 width=196) (actual time=1267.595..1282.422 rows=10432 loops=1)'},
 {'QUERY PLAN': '  Group Key: app_prof.id'},
 {'QUERY PLAN': '  ->  Gather Merge  (cost=31623.73..31633.90 rows=78 width=196) (actual time=1267.587..1276.091 rows=10576 loops=1)'},
 {'QUERY PLAN': '        Workers Planned: 2'},
 {'QUERY PLAN': '        Workers Launched: 2'},
 {'QUERY PLAN': '        ->  Partial GroupAggregate  (cost=30623.71..30624.88 rows=39 width=196) (actual time=1262.297..1266.791 rows=3525 loops=3)'},
 {'QUERY PLAN': '              Group Key: app_prof.id'},
 {'QUERY PLAN': '              ->  Sort  (cost=30623.71..30623.81 rows=39 width=1684) (actual time=1262.287..1262.623 rows=9285 loops=3)'},
 {'QUERY PLAN': '                    Sort Key: app_prof.id'},
 {'QUERY PLAN': '                    Sort Method: quicksort  Memory: 2311kB'},
 {'QUERY PLAN': '                    Worker 0:  Sort Method: quicksort  Memory: 2348kB'},
 {'Q

In [387]:
import pandas as pd

data = []
for item in aplicantes_filtrados:
    line = item['QUERY PLAN']
    if 'cost=' in line and 'actual time=' in line:
        operation = line.split('  ')[-1].strip()  # Operación
        cost = line.split('cost=')[1].split(' ')[0].split('..')
        actual_time = line.split('actual time=')[1].split(' ')[0].split('..')
        rows = line.split('rows=')[1].split(' ')[0] if 'rows=' in line else None
        data.append({
            'Operación': operation,
            'Costo Inicial': float(cost[0]),
            'Costo Final': float(cost[1]),
            'Tiempo Inicial (ms)': float(actual_time[0]),
            'Tiempo Final (ms)': float(actual_time[1]),
            'Filas Procesadas': int(rows) if rows else None
        })

# Crear un DataFrame
df = pd.DataFrame(data)

df

,Operación,Costo Inicial,Costo Final,Tiempo Inicial (ms),Tiempo Final (ms),Filas Procesadas
0,(cost=31623.73..31636.21 rows=94 width=196) (a...,31623.73,31636.21,1267.595,1282.422,94
1,(cost=31623.73..31633.90 rows=78 width=196) (a...,31623.73,31633.90,1267.587,1276.091,78
2,(cost=30623.71..30624.88 rows=39 width=196) (a...,30623.71,30624.88,1262.297,1266.791,39
3,(cost=30623.71..30623.81 rows=39 width=1684) (...,30623.71,30623.81,1262.287,1262.623,39
4,(cost=20209.84..30622.68 rows=39 width=1684) (...,20209.84,30622.68,176.909,1258.362,39
5,(cost=16326.13..25432.86 rows=7822 width=1595)...,16326.13,25432.86,160.367,223.992,7822
6,(cost=16323.55..25408.50 rows=7822 width=1083)...,16323.55,25408.50,160.289,211.136,7822
7,(cost=0.00..7648.03 rows=370903 width=8) (actu...,0.00,7648.03,0.003,14.321,370903
8,(cost=16306.29..16306.29 rows=1381 width=1075)...,16306.29,16306.29,160.238,160.247,1381
9,(cost=13952.18..16306.29 rows=1381 width=1075)...,13952.18,16306.29,148.800,157.391,1381
